In [10]:
import re
from Graph import *
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

global_g_amazon = Load_Graph('./datasets-RW/com-amazon.ungraph.txt')


Loading graph from ./datasets-RW/com-amazon.ungraph.txt
Graph size: E = 925872, V = 334863


In [11]:
from Random_Walk import *
import json

seed_node_id = 21 ### amazon graph has no node named 70. Therefore, changes to 21
BFS_depth = 4
top_c = 50
max_length = 8
alpha = 0.95


Clear_Counter(global_g_amazon)
Random_Walk_Seed_Local( global_g = global_g_amazon, sub_g = global_g_amazon, start_step = 0, seed_node_id = seed_node_id, \
                        top_c = top_c, alpha = alpha, max_length = max_length, max_itr_seed = 1000000 )
ground_truth = Compute_Score_Rank( global_g_amazon, top_c, seed_node_id, alpha )
print("ground_truth")
print(ground_truth)

for node in ground_truth:
    print("Node: %d" % node)
    print(global_g_amazon.nodes[node].counters)


ground_truth
[421307, 152202, 184384, 340707, 320969, 198272, 536600, 234603, 528460, 160012, 384493, 297444, 50332, 543461, 62691, 21, 166829, 505568, 416560, 17723, 538669, 114681, 231335, 129852, 99962, 546428, 456289, 444122, 167231, 256332, 101545, 410119, 476746, 545557, 319239, 547758, 157391, 122698, 274741, 90065, 96553, 216871, 231263, 545216, 416561, 544376, 251900, 207520, 107957, 229702]
Node: 421307
{0: 100854, 2: 37430, 6: 11660, 4: 19078, 3: 3964, 1: 4646, 5: 3768, 7: 3508}
Node: 152202
{0: 100580, 5: 2330, 2: 28780, 4: 11086, 3: 1894, 6: 5462, 7: 1868}
Node: 184384
{0: 100184, 2: 28454, 4: 11422, 5: 2250, 6: 5662, 7: 1970, 3: 1974}
Node: 340707
{4: 15194, 0: 99948, 3: 7356, 1: 7456, 2: 33552, 6: 8676, 5: 5950, 7: 4656}
Node: 320969
{0: 100004, 2: 33978, 1: 7552, 4: 15032, 3: 7216, 5: 5748, 7: 4606, 6: 8506}
Node: 198272
{0: 100396, 2: 21188, 4: 8734, 6: 4864, 5: 2096, 3: 1904, 7: 1936}
Node: 536600
{2: 71640, 0: 99822, 4: 44758, 5: 3006, 7: 3500, 6: 27676, 3: 1988}
Nod

In [79]:
seed_node_id = 21
BFS_depth = 2
max_length = 2
global_g = global_g_amazon
print("=== Computing seed node " + str(seed_node_id))
Clear_Counter(global_g)

sub_g_full = BFS_Sub_Graph(global_g, seed_node_id, 6)


# first level 2-depth BFS
print("-- First BFS with depth %d length %d --" % (BFS_depth, max_length))
sub_g = BFS_Sub_Graph(global_g, seed_node_id, BFS_depth)
print("Sub-graph size: V = %d, E = %d" % (sub_g.node_cnt, sub_g.edge_cnt))

Clear_Counter( sub_g )
Random_Walk_Seed_Local( global_g = global_g, sub_g = sub_g, start_step = 0, seed_node_id = seed_node_id, \
                        top_c = top_c, alpha = alpha, max_length = max_length, max_itr_seed = 40000 )
top_nodes_1st = Compute_Score_Rank( global_g, top_c, seed_node_id, alpha )

similarity = get_same_ele_cnt( ground_truth, top_nodes_1st ) / top_c
print("similarity: %.2f" % (similarity))

print("Top nodes:")
print(top_nodes_1st)
print("Ground truth:")
print(ground_truth)


print("Missing:")
for node in ground_truth:
    if node not in top_nodes_1st:
        print(node)
print("###")

for node in top_nodes_1st:
    print("Node: %d" % node)
    print(global_g.nodes[node].counters)


=== Computing seed node 21
-- First BFS with depth 2 length 2 --
Sub-graph size: V = 122, E = 278
similarity: 0.84
Top nodes:
[421307, 152202, 198272, 184384, 234603, 297444, 320969, 340707, 528460, 50332, 62691, 384493, 536600, 21, 160012, 166829, 543461, 505568, 416560, 538669, 17723, 114681, 231335, 99962, 444122, 476746, 546428, 545557, 547758, 256332, 96553, 231263, 129852, 205628, 101545, 251900, 167231, 61658, 416561, 207520, 229702, 456289, 127175, 61657, 122698, 107957, 237452, 543148, 285935, 333844]
Ground truth:
[421307, 152202, 184384, 340707, 320969, 198272, 536600, 234603, 528460, 160012, 384493, 297444, 50332, 543461, 62691, 21, 166829, 505568, 416560, 17723, 538669, 114681, 231335, 129852, 99962, 546428, 456289, 444122, 167231, 256332, 101545, 410119, 476746, 545557, 319239, 547758, 157391, 122698, 274741, 90065, 96553, 216871, 231263, 545216, 416561, 544376, 251900, 207520, 107957, 229702]
Missing:
410119
319239
157391
274741
90065
216871
545216
544376
###
Node: 42130

In [80]:


### To continue with boundary nodes 
# collect boundary nodes
boundary_nodes = []
def create_boundary_nodes(boundary_nodes, sub_g):
    for node_id in sub_g.nodes:
        node = sub_g.nodes[node_id]
        if node.depth != 2:
            continue
        # if 2 not in node.counters:
        #     continue
        # if node.counters[3] < 5:
        #     continue
        boundary_nodes.append(node_id)
        #print("Node: %d (depth %d)" % (node_id, node.depth))
        #print(node.counters)
    return boundary_nodes
boundary_nodes = create_boundary_nodes([], sub_g)
print(len(boundary_nodes))


101


In [81]:


# for node in top_nodes_1st:
#     if global_g.nodes[node].counters:
#         print("Node: " + str(node), end=" ")
#         print(global_g.nodes[node].counters, end=" ")
#         print(global_g.nodes[node].s_u_a_score)

# boundary_nodes = [25]

print("-- Second BFS with depth %d length %d --" % (BFS_depth, max_length))
# select nodes to apply second BFS

#for seed_node_id in boundary_nodes:
print(boundary_nodes)
new_boundary_nodes = []
number_of_bfs = 2
dpt = 1
for k in range(number_of_bfs):
    number_of_boundary_nodes = len(boundary_nodes)
    for i in range(number_of_boundary_nodes):

        seed_node_id = boundary_nodes[i]
        #print("seed_node_id", seed_node_id)
        #print("global_g.nodes[seed_node_id].counters[]", seed_node_id, global_g.nodes[seed_node_id].counters)
        if dpt in global_g.nodes[seed_node_id].counters:
            itr_cnt = int(global_g.nodes[seed_node_id].counters[dpt])
            print("=== Computing seed node %d for %d iterations" % (seed_node_id, itr_cnt))
            sub_g = BFS_Sub_Graph(global_g, seed_node_id, BFS_depth)

            new_boundary_nodes.extend(create_boundary_nodes([], sub_g))
            Clear_Counter( sub_g )

            Random_Walk_Seed_Local( global_g = global_g, sub_g = sub_g, start_step = BFS_depth, seed_node_id = seed_node_id, \
                                top_c = top_c, alpha = alpha, max_length = 2, max_itr_seed = itr_cnt )


            # if 2 not in global_g.nodes[seed_node_id].counters:
            #     continue
            # itr_cnt = int(global_g.nodes[seed_node_id].counters[2])
            # print("=== Computing seed node %d for %d iterations" % (seed_node_id, itr_cnt))
            # sub_g = BFS_Sub_Graph(global_g, seed_node_id, BFS_depth)
            # Clear_Counter( sub_g )

            # Random_Walk_Seed_Local( global_g = global_g, sub_g = sub_g, start_step = BFS_depth+1, seed_node_id = seed_node_id, \
            #                     top_c = top_c, alpha = alpha, max_length = 2, max_itr_seed = itr_cnt )


            top_nodes = Compute_Score_Rank( global_g, top_c, seed_node_id, alpha )
            similarity = get_same_ele_cnt( ground_truth, top_nodes ) / top_c
            print("similarity: %.2f" % (similarity))
    BFS_depth += 2
    #print("new_boundary_nodes", new_boundary_nodes)
    #boundary_nodes = new_boundary_nodes
    boundary_nodes = []
    node_score_pair = []
    for node in new_boundary_nodes:
        node_score_pair.append([node, sum(global_g.nodes[node].counters.values())])
    node_score_pair.sort(key = lambda x: x[1], reverse=True)
    #print("node_score_pair", node_score_pair)

    for i in range(20):
        boundary_nodes.append(node_score_pair[i][0])
    new_boundary_nodes = []
    dpt += 2
    print("boundary_nodes", boundary_nodes)
    print("dpt", dpt)



-- Second BFS with depth 2 length 2 --
[546264, 61657, 127175, 416560, 83709, 546238, 385667, 488323, 332165, 237452, 333844, 67739, 406560, 548387, 346020, 547112, 101545, 544689, 22331, 182207, 167231, 229702, 544838, 314571, 415310, 152399, 313171, 231519, 547300, 285935, 266995, 545523, 208377, 546428, 543461, 231335, 476746, 160012, 384493, 256332, 319239, 99962, 251900, 8076, 336912, 497426, 276502, 260639, 216871, 368424, 215612, 90065, 302178, 53609, 543148, 416561, 17401, 205628, 231263, 227969, 443140, 410119, 354953, 533770, 109968, 191137, 363426, 446280, 541903, 333654, 510435, 542824, 545778, 357879, 544376, 207520, 156197, 546597, 86193, 41528, 547517, 35264, 155711, 458337, 21350, 310665, 85666, 283570, 160310, 375015, 456289, 122698, 107957, 129852, 444122, 61658, 519820, 274741, 97974, 157391, 99709]
=== Computing seed node 546264 for 824 iterations
similarity: 0.84
=== Computing seed node 61657 for 858 iterations
similarity: 0.84
=== Computing seed node 127175 for 82

similarity: 0.88
=== Computing seed node 21 for 890 iterations
similarity: 0.88
=== Computing seed node 21 for 890 iterations
similarity: 0.88
=== Computing seed node 21 for 890 iterations
similarity: 0.90
=== Computing seed node 21 for 890 iterations
similarity: 0.90
=== Computing seed node 21 for 890 iterations
similarity: 0.90
=== Computing seed node 21 for 890 iterations
similarity: 0.90
=== Computing seed node 21 for 890 iterations
similarity: 0.90
=== Computing seed node 21 for 890 iterations
similarity: 0.90
=== Computing seed node 21 for 890 iterations
similarity: 0.90
=== Computing seed node 21 for 890 iterations
similarity: 0.90
boundary_nodes [416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560, 416560]
dpt 5


In [56]:
### this cell define a function to merge 2 subgraphs
def merge_subgraphs(subgraph1, subgraph2):
    merged_subgraph = Graph()
    for node_id in subgraph1.nodes:
        for nei in subgraph1.nodes[node_id].neighbors:
            merged_subgraph.add_edge(node_id,nei)
    
    for node_id in subgraph2.nodes:
        for nei in subgraph2.nodes[node_id].neighbors:
            merged_subgraph.add_edge(node_id,nei)
                
    return merged_subgraph

In [ ]:

### this cell is trying to write bram content to a txt file (the original random walk; not the diffusion)
number_of_seed = 20
#for i in range(number_of_seed):
#    print(boundary_nodes[i])
aggr_node = []
aggr_subgraphs = Graph()
for i in range(number_of_seed):
    seed_node_id=boundary_nodes[i]
    sub_g_i = BFS_Sub_Graph(global_g, seed_node_id, BFS_depth)
    aggr_subgraphs = merge_subgraphs(aggr_subgraphs, sub_g_i)
    
    for node_id in sub_g_i.nodes:
        #print(node_id)        
        #print(sub_g_i.nodes[node_id].neighbors)
        if node_id not in aggr_node: aggr_node.append(node_id)
        for nei in sub_g_i.nodes[node_id].neighbors:
            if nei not in aggr_node: aggr_node.append(nei)
aggr_node.sort()
aggr_node_dict = {}
aggr_node_dict_reverse = {}
for idx, val in enumerate(aggr_node):
    aggr_node_dict[idx+1] = val ### start from node 1; not 0 
    aggr_node_dict_reverse[val] = idx+1 ### start from node 1; not 0 
#print(aggr_node_dict)
f = open("bram_content.txt", "w")
f.write("0 \n")
for i in range(number_of_seed):
    seed_node_id=boundary_nodes[i]
    #print(seed_node_id, aggr_node_dict_reverse[seed_node_id])
    f.write(str(aggr_node_dict_reverse[seed_node_id]) + "\n") ### write seed nodes(its hashing number instead of real node number) to bram 

number_of_nodes = len(aggr_node)
nei_addr_table_offset = number_of_seed + 1
nei_table_offset = number_of_nodes*2 + nei_addr_table_offset ### 387 nodes in aggr_graph, 21 is nei_addr_table_offset
curr_nei_table_offset = nei_table_offset
print("total number of nodes are", len(aggr_node))
for idx, node_id in enumerate(aggr_node):
    number_of_neighbour = len(aggr_subgraphs.nodes[node_id].neighbors)
    f.write(str(curr_nei_table_offset) + "\n") 
    f.write(str(curr_nei_table_offset+number_of_neighbour-1) + "\n") 
    curr_nei_table_offset += number_of_neighbour
    
for idx, node_id in enumerate(aggr_node):
    for nei in aggr_subgraphs.nodes[node_id].neighbors:
        f.write(str(aggr_node_dict_reverse[nei]) + "\n") 
    #print(aggr_subgraphs.nodes[node_id].neighbors)

print(aggr_node_dict[80])
print(aggr_node_dict[224])



print(aggr_subgraphs.nodes[582].neighbors)
print(aggr_subgraphs.nodes[1624].neighbors)
f.close()

In [101]:
### this cell is printing out the diffusion score ground truth 
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")

lap = 7
node_num = 550000  ### in amazon graph, some node numbers are not present, e.g there is no node 70; 550000 is the max number of node id 
initial_score = [0] * node_num
#initial_score[0] = 10000
#initial_score[499] = 10000
#initial_score[999] = 10000
initial_score[20] = 1000000
score = [0] * node_num
score = initial_score
new_score = [0] * node_num

print("number of node", len(global_g.nodes))
print("start Time =", current_time)
for _ in range(lap):
    
    for node in global_g.nodes: ## 
        degree = len(global_g.nodes[node].neighbors)
        #print("degree", degree)
        for nei in global_g.nodes[node].neighbors:
            #print("nei", nei, node, len(new_score), len(new_score))
            new_score[int(nei-1)] += score[int(node-1)]//degree
            #pass
    score = new_score
    new_score = [0]*node_num
    
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("end Time =", current_time)

#score.sort(reverse=True)
#print(score[:20])



number of node 334863
start Time = 19:38:28
end Time = 19:38:43


In [24]:

### this cell is printing out bram content to the txt files (diffusion_rw dual)
bram_subgraph_1 = "bram_cora_subgraph_1.txt"
bram_subgraph_2 = "bram_cora_subgraph_2.txt"


bram_score_1 = "bram_cora_score_table_1.txt"
bram_score_2 = "bram_cora_score_table_2.txt"


half_node = node_num//2

offset = half_node * 2
first_neighbour_addr = offset
f1 = open(bram_subgraph_1, 'w')
f2 = open(bram_subgraph_2, "w")

f3 = open(bram_score_1, 'w')
f4 = open(bram_score_2, "w")

for node in range(1, half_node+1):
    
    degree = len(global_g.nodes[node].neighbors)
    last_neighbour_addr = first_neighbour_addr + degree - 1
    f1.write("%s " % hex(first_neighbour_addr).replace("0x", ""))  
    f1.write("\n")
    f1.write("%s " % hex(last_neighbour_addr).replace("0x", "")) 
    f1.write("\n")
    first_neighbour_addr = last_neighbour_addr + 1
    
offset = (node_num-half_node) * 2
first_neighbour_addr = offset

for node in range(1, half_node+1):
    for nei in global_g.nodes[node].neighbors:
        f1.write("%s " % hex(nei).replace("0x", ""))  
        f1.write("\n")
f1.close()

for node in range(half_node+1, node_num+1):
    degree = len(global_g.nodes[node].neighbors)
    last_neighbour_addr = first_neighbour_addr + degree - 1
    f2.write("%s " % hex(first_neighbour_addr).replace("0x", ""))  
    f2.write("\n")
    f2.write("%s " % hex(last_neighbour_addr).replace("0x", "")) 
    f2.write("\n")
    first_neighbour_addr = last_neighbour_addr + 1
    
for node in range(half_node+1, node_num+1):
    for nei in global_g.nodes[node].neighbors:
        f2.write("%s " % hex(nei).replace("0x", ""))  
        f2.write("\n")
f2.close()


for i in range(half_node):
    f3.write("%s " % hex(initial_score[i]).replace("0x", "")) ## verilog test bench only hex or binary only  
    f3.write("\n")
    f3.write("%d " % 0)  
    f3.write("\n")
    
f3.close()
    
for i in range(half_node, node_num):
    f4.write("%s " % hex(initial_score[i]).replace("0x", ""))  
    f4.write("\n")
    f4.write("%d " % 0)  
    f4.write("\n") 
    
f4.close()

In [35]:

### this cell is printing out bram content to the txt files (diffusion_rw quad)

bram_quad_subgraph_1 = "bram_quad_cora_subgraph_1.txt"
bram_quad_subgraph_2 = "bram_quad_cora_subgraph_2.txt"
bram_quad_subgraph_3 = "bram_quad_cora_subgraph_3.txt"
bram_quad_subgraph_4 = "bram_quad_cora_subgraph_4.txt"


bram_quad_score_1 = "bram_quad_cora_score_table_1.txt"
bram_quad_score_2 = "bram_quad_cora_score_table_2.txt"
bram_quad_score_3 = "bram_quad_cora_score_table_3.txt"
bram_quad_score_4 = "bram_quad_cora_score_table_4.txt"


quarter = node_num//4
half_node = node_num//2
third_quarter = node_num//4 * 3;


f1 = open(bram_quad_subgraph_1, 'w')
f2 = open(bram_quad_subgraph_2, "w")
f3 = open(bram_quad_subgraph_3, 'w')
f4 = open(bram_quad_subgraph_4, "w")


f5 = open(bram_quad_score_1, 'w')
f6 = open(bram_quad_score_2, "w")
f7 = open(bram_quad_score_3, 'w')
f8 = open(bram_quad_score_4, "w")

for node in range(1, node_num+1):
    if node == 1:
        f = f1
        offset = quarter * 2
        first_neighbour_addr = offset    
    elif node == quarter+1:
        f = f2
        offset = (half_node-quarter) * 2
        first_neighbour_addr = offset    
    elif node == half_node+1:
        f = f3
        offset = (third_quarter-half_node) * 2
        first_neighbour_addr = offset    
    elif node == third_quarter+1:
        f = f4
        offset = (node_num-third_quarter) * 2
        first_neighbour_addr = offset 
        
    degree = len(global_g.nodes[node].neighbors)
    last_neighbour_addr = first_neighbour_addr + degree - 1
    f.write("%s " % hex(first_neighbour_addr).replace("0x", ""))  
    #f.write("%s " % first_neighbour_addr)  
    f.write("\n")
    f.write("%s " % hex(last_neighbour_addr).replace("0x", "")) 
    #f.write("%s " % last_neighbour_addr)
    f.write("\n")
    first_neighbour_addr = last_neighbour_addr + 1
    
offset = (node_num-half_node) * 2
first_neighbour_addr = offset

for node in range(1, node_num+1):
    if node == 1:
        f = f1
    elif node == quarter+1:
        f = f2
    elif node == half_node+1:
        f = f3
    elif node == third_quarter+1: 
        f = f4
        
    for nei in global_g.nodes[node].neighbors:
        f.write("%s " % hex(nei).replace("0x", ""))  
        #f.write("%s " % nei)
        f.write("\n")
f1.close()
f2.close()
f3.close()
f4.close()




for i in range(node_num):
    
    if i == 0:
        f = f5 
    elif i == quarter:
        f = f6
    elif i == half_node:
        f = f7
    elif i == third_quarter:
        f = f8
        
        
    f.write("%s " % hex(initial_score[i]).replace("0x", "")) ## verilog test bench only hex or binary only  
    f.write("\n")
    f.write("%d " % 0)  
    f.write("\n")
    
f5.close()
f6.close()
f7.close()
f8.close()
    


In [37]:
### this cell is printing out bram content to the txt files (diffusion_rw octa)


bram_octa_subgraph_1 = "bram_octa_cora_subgraph_1.txt"
bram_octa_subgraph_2 = "bram_octa_cora_subgraph_2.txt"
bram_octa_subgraph_3 = "bram_octa_cora_subgraph_3.txt"
bram_octa_subgraph_4 = "bram_octa_cora_subgraph_4.txt"
bram_octa_subgraph_5 = "bram_octa_cora_subgraph_5.txt"
bram_octa_subgraph_6 = "bram_octa_cora_subgraph_6.txt"
bram_octa_subgraph_7 = "bram_octa_cora_subgraph_7.txt"
bram_octa_subgraph_8 = "bram_octa_cora_subgraph_8.txt"



bram_octa_score_1 = "bram_octa_cora_score_table_1.txt"
bram_octa_score_2 = "bram_octa_cora_score_table_2.txt"
bram_octa_score_3 = "bram_octa_cora_score_table_3.txt"
bram_octa_score_4 = "bram_octa_cora_score_table_4.txt"
bram_octa_score_5 = "bram_octa_cora_score_table_5.txt"
bram_octa_score_6 = "bram_octa_cora_score_table_6.txt"
bram_octa_score_7 = "bram_octa_cora_score_table_7.txt"
bram_octa_score_8 = "bram_octa_cora_score_table_8.txt"

octa = 338 ## 2078 // 8
quarter = octa*2
half_node = octa*4
third_quarter = octa*6


f1 = open(bram_octa_subgraph_1, 'w')
f2 = open(bram_octa_subgraph_2, "w")
f3 = open(bram_octa_subgraph_3, 'w')
f4 = open(bram_octa_subgraph_4, "w")
f5 = open(bram_octa_subgraph_5, 'w')
f6 = open(bram_octa_subgraph_6, "w")
f7 = open(bram_octa_subgraph_7, 'w')
f8 = open(bram_octa_subgraph_8, "w")

f9 = open(bram_octa_score_1, 'w')
f10 = open(bram_octa_score_2, "w")
f11 = open(bram_octa_score_3, 'w')
f12 = open(bram_octa_score_4, "w")
f13 = open(bram_octa_score_5, 'w')
f14 = open(bram_octa_score_6, "w")
f15 = open(bram_octa_score_7, 'w')
f16 = open(bram_octa_score_8, "w")

for node in range(1, node_num+1):
    if node == 1:
        f = f1
        offset = octa * 2
        first_neighbour_addr = offset    
    elif node == octa+1:
        f = f2
        offset = octa * 2
        first_neighbour_addr = offset    
    elif node == quarter+1:
        f = f3
        offset = octa * 2
        first_neighbour_addr = offset    
    elif node == octa+quarter+1:
        f = f4
        offset = octa * 2
        first_neighbour_addr = offset
    elif node == quarter*2 + 1:
        f = f5
        offset = octa * 2
        first_neighbour_addr = offset
    elif node == quarter*2 + octa+ 1:
        f = f6
        offset = octa * 2
        first_neighbour_addr = offset
    elif node == quarter*3 + 1:
        f = f7
        offset = octa * 2
        first_neighbour_addr = offset
    elif node == quarter*3 + octa +1:
        f = f8
        offset = (node_num-octa*7) * 2
        first_neighbour_addr = offset        
    
    degree = len(global_g.nodes[node].neighbors)
    last_neighbour_addr = first_neighbour_addr + degree - 1
    f.write("%s " % hex(first_neighbour_addr).replace("0x", ""))  
    #f.write("%s " % first_neighbour_addr)  
    f.write("\n")
    f.write("%s " % hex(last_neighbour_addr).replace("0x", "")) 
    #f.write("%s " % last_neighbour_addr)
    f.write("\n")
    first_neighbour_addr = last_neighbour_addr + 1
    
offset = (node_num-half_node) * 2
first_neighbour_addr = offset

for node in range(1, node_num+1):
    if node == 1:
        f = f1
    elif node == octa+1:
        f = f2
    elif node == quarter+1:
        f = f3
    elif node == octa+quarter+1:
        f = f4
    elif node == quarter*2 + 1:
        f = f5
    elif node == quarter*2 + octa+ 1:
        f = f6
    elif node == quarter*3 + 1:
        f = f7
    elif node == quarter*3 + octa +1:
        f = f8

    for nei in global_g.nodes[node].neighbors:
        f.write("%s " % hex(nei).replace("0x", ""))  
        #f.write("%s " % nei)
        f.write("\n")
f1.close()
f2.close()
f3.close()
f4.close()
f5.close()
f6.close()
f7.close()
f8.close()




for i in range(node_num):
    
    if i == 0:
        f = f9 
    elif i == octa:
        f = f10
    elif i == octa*2:
        f = f11
    elif i == octa*3:
        f = f12
    elif i == octa*4:
        f = f13
    elif i == octa*5:
        f = f14
    elif i == octa*6:
        f = f15
    elif i == octa*7:
        f = f16

        
    f.write("%s " % hex(initial_score[i]).replace("0x", "")) ## verilog test bench only hex or binary only  
    f.write("\n")
    f.write("%d " % 0)  
    f.write("\n")
    
f9.close()
f10.close()
f11.close()
f12.close()
f13.close()
f14.close()
f15.close()
f16.close()